In [28]:
from scipy.io import netcdf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import netCDF4 as nc
import matplotlib as mpl
import xarray as xr
import geopandas as gpd

In [1]:
# install modul yang diperlukan
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import numpy as np

In [30]:
#script ini digunakan untuk melakukan re-gridding menjadi 0.50 derajat

In [6]:
ds = xr.open_dataset("/Users/roniabuhanif/Downloads/BMKG/data_skripsi/data_bu_azizah/co2/co2.nc") #membukan file .nc yang diperlukan
ds

<xarray.Dataset>
Dimensions:    (longitude: 6, latitude: 4, time: 60)
Coordinates:
  * longitude  (longitude) float32 110.9 111.7 112.4 113.2 113.9 114.7
  * latitude   (latitude) float32 -6.63 -7.38 -8.13 -8.88
  * time       (time) datetime64[ns] 2016-01-01 2016-02-01 ... 2020-12-01
Data variables:
    co2        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-12-15 03:39:02 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
latii = [-6.294]
lonii = [110.915]

new_lon = np.arange(ds.longitude[0], ds.longitude[-1], 0.05)  # 0.4 to 0.05
new_lat = np.arange(ds.latitude[-1], ds.latitude[0], 0.05)
ds = ds.interp(latitude=new_lat, longitude=new_lon)

In [34]:
import math

In [35]:
# mengubah data menjadi data tipe dataframa
df = ds.to_dataframe()
# konversi kg/kg menjadi ppm
# 1. kg/kg to ppb = divided by 1x10^-9
# 2. ppb to ppm = multiply by 0.001
df ["co2"] = (df["co2"]/0.000000001)*0.001
# menyimpan dataframe menjadi .csv
df.to_csv("co2_satuan.csv")

In [10]:
# kg/kg to ppm multiply by 10^6
df = ds.to_dataframe()
df ["co2"] = (df["co2"]*10000*100000)/0.001
df.to_csv("co2_satuan_sda_2.csv")


In [10]:
import geopandas as gpd
import pandas as pd

In [11]:
# Load shapefile into a GeoDataFrame
gdf_shapefile = gpd.read_file('/Users/roniabuhanif/Downloads/BMKG/data_skripsi/peta_jatim/kabupaten.shp')

In [12]:
gdf_shapefile.crs

In [13]:
gdf_shapefile.head(5)

,IDKABKOTA,COUNT,urutan_kab,KEPADATAN,SEXRATIO,TFR,CBR,CPR,ALH3PLUS,UKP,...,KS1,PRAKS_KS1,IPM,JAMBANSEP,SAKIT,AIRBERSIH,LPP,ASFR_20_24,ASFR,geometry
0,BANGKALAN,283,26,696,91.48,1.819,12.42,36.66,35.01,18.64,...,35.20,58.39,64.52,25.68,17.93,97.22,1.21,None,None,"POLYGON ((112.68201 -7.08612, 112.68037 -7.083..."
1,BANYUWANGI,227,10,432,99.08,2.077,14.29,61.76,25.89,19.27,...,20.89,41.69,68.81,54.49,32.71,88.68,0.45,None,None,"MULTIPOLYGON (((113.90852 -8.58587, 113.90841 ..."
2,BATU,24,38,942,101.28,2.119,14.59,69.07,20.81,20.50,...,18.29,29.86,74.35,78.21,39.54,99.18,1.25,None,None,"POLYGON ((112.48183 -7.86983, 112.48473 -7.867..."
3,BOJONEGORO,432,22,523,97.83,1.891,12.94,73.54,21.09,18.95,...,16.59,68.66,66.84,40.79,24.11,95.85,0.38,None,None,"POLYGON ((112.07846 -7.35360, 112.08035 -7.358..."
4,BONDOWOSO,211,11,469,94.97,1.997,13.71,67.03,21.92,17.08,...,22.44,64.54,62.79,22.12,33.68,88.40,0.68,None,None,"POLYGON ((114.06098 -8.12161, 114.05655 -8.125..."


In [14]:
# Load CSV
df_csv = pd.read_csv('co2_satuan.csv')

In [15]:
df_csv.head(54000)

,time,latitude,longitude,co2
0,2016-01-01,-8.88,110.910004,610.471237
1,2016-01-01,-8.88,110.960004,610.641677
2,2016-01-01,-8.88,111.010004,610.812117
3,2016-01-01,-8.88,111.060004,610.982557
4,2016-01-01,-8.88,111.110004,611.152997
...,...,...,...,...
53995,2017-04-01,-6.68,114.410004,609.816734
53996,2017-04-01,-6.68,114.460004,609.790856
53997,2017-04-01,-6.68,114.510004,609.764979
53998,2017-04-01,-6.68,114.560004,609.739102


sampai sini


In [16]:
# Convert DataFrame to GeoDataFrame
geometry = gpd.points_from_xy(df_csv['longitude'], df_csv['latitude'])
gdf_csv = gpd.GeoDataFrame(df_csv, geometry=geometry)


In [17]:
# spatial join
result = gpd.sjoin(gdf_shapefile, gdf_csv, how='inner', predicate='within')
dissolved_result = result.dissolve(by= 'IDKABKOTA', aggfunc=['min', 'max', 'mean'])
dissolved_result

,geometry,"(COUNT, min)","(COUNT, max)","(COUNT, mean)","(urutan_kab, min)","(urutan_kab, max)","(urutan_kab, mean)","(KEPADATAN, min)","(KEPADATAN, max)","(KEPADATAN, mean)",...,"(time, mean)","(latitude, min)","(latitude, max)","(latitude, mean)","(longitude, min)","(longitude, max)","(longitude, mean)","(co2, min)","(co2, max)","(co2, mean)"
IDKABKOTA,,,,,,,,,,,,,,,,,,,,,
